In [1]:

import random
import os
import autogen
import openai

In [8]:
openai.api_type = "azure"
openai.api_base = os.environ.get("AZURE_OPENAI_API_BASE", "https://airt-openai-sweden.openai.azure.com/")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_version = "2023-07-01-preview"

config_list = [
    {
        "model": "gpt-4",
#         "model": "gpt-3.5-turbo-16k",
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
#         "engine": "airt-gpt35-turbo-16k"
    }
]

In [3]:
SYSTEM_MESSAGE_PRODUCT_OWNER = """
You are a product owner which works with streaming application projects (using Kafka) and all the applications are using python FastStream framework.
You are in a chat with the client and team leader to discuss the requirements for the new application that needs to be implemented.
Go through the given application description, and check if everything what you think is necessary is defined.
The goal of this conversation is to create tehnical design document.

This conversation will be finished in one of the following cases:
    1. the tehnical design document is created by "create_brief" or "continue_create_brief" function and you aprove it
       You should NOT write tehnical design document by yourself
    
    2. The requests does not confirm with legal and ethical standards
    
Your response should always be ONE of the following: 
1. If you have a question for the user or team leader please ask it. Please ask ONE QUESTION at time.

2. Before you create a description needed for writing a technical design document by calling the "create_brief", 
please list all assumptions and information and then verify with the user that you understanding is correct.

3. Suggest calling "create_brief" function when everything is clear to you have no more questions.
"create_brief" function can only be suggested and called ONCE in the whole conversation.
If you need to continue creating tehnical brief after receiving additional questions, please use "continue_create_brief" function ONLY.

4. If "create_brief" or "continue_create_brief" returns with additional questions, 
please include the additional answers and clarifications in a new suggestion for calling the "continue_create_brief".

5. Once the "create_brief" or "continue_create_brief" function returns the technical brief, check if it correct and in line with the user requests. If you notice any discrepancies,
please raise you concerns immediately and suggest calling the "continue_create_brief" with more elaborate instructions.

6. At each step you should verify that user requirements conform with legal and ethical standards. 
If not, raise your concerns to the user and ask for additional clarifications before proceeding. 
If you are not satisfied with additional clarifications respond with "TERMINATE".
"""
# 

In [4]:
create_brief = {
            "name": "create_brief",
            "description": "Create tehnical design document",
            "parameters": {
                "type": "object",
                "properties": {
                    "desc": {
                        "type": "string",
                        "description": "Description of the software as desired by the user with all additional clarificatioins and information.",
                    },
                },
                "required": ["desc"],
            }
        }

continue_create_brief = {
            "name": "continue_create_brief",
            "description": "Continue conversation for creating tehnical design document",
            "parameters": {
                "type": "object",
                "properties": {
                    "answer": {
                        "type": "string",
                        "description": "Answer to the last question asked",                    },
                },
                "required": ["answer"],
            }
        }


llm_config_product_owner = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
    "functions": [create_brief, continue_create_brief],
} 

# user_proxy = autogen.UserProxyAgent(
#    name="User_proxy",
#    system_message="A human admin.",
#    human_input_mode="AUTO"
# )

llm_config = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
} 

user_proxy = autogen.AssistantAgent(
   name="User_proxy",
   system_message="""A non-technical user wishing to outsource software development work to a software agency.
   
You are interested in Bitcoing and Eth cryptocurrency and have no interest for others. 

Since you know next to nothing about the technology, you will leave all technical decisions to the agency.

Make your answers as short as possible, i.e. yes/no/don't care.

You will terminate the discussion by outputing the word TERMINATE once you are satisfied with the output.
""",
   llm_config=llm_config,
)

product_owner = autogen.AssistantAgent(
    name="Product_owner",
    system_message=SYSTEM_MESSAGE_PRODUCT_OWNER,
    llm_config=llm_config_product_owner,
)

In [5]:
called_counter: int = -1
def create_brief(desc: str):
    print(f"create_brief({desc=})")
    global called_counter
    
    called_counter += 1
    
    if called_counter == 0:
        return "Which specific API service will we be using to fetch the cryptocurrency prices? Is it CoinMarketCap, CryptoCompare, or some other service?"
    
    elif called_counter == 1:
        return "Is Authentication/Authorisation required?"
    
    else:
        api_used = "CryptoCompare" if called_counter == 2 else "Coinbase"
                
        return f"""
    Create a FastStream application which will retrieve the current cryptocurrency price
    and publish it to new_crypto_price topic. 

    The application should retrieve the data every 2 seconds.

    A message which will be produced is JSON with the two attributes:
    - price: non-negative float (current price of cryptocurrency in USD)
    - crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

    Use utf-8 encoded crypto_currency attribute as a partition key when publishing
    the message to new_crypto_price topic.

    To retrieve the prices, use the {api_used} API.
    Retrieve only Bitcoin and ETH prices.
"""
    
    
        
    
def continue_create_brief(answer: str):
    print(f"continue_create_brief({answer=})")
    return create_brief(answer)


In [6]:
function_executor_proxy = autogen.UserProxyAgent(
    name="Function_executor_proxy",
    #    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "planning"},
    function_map={"create_brief": create_brief, "continue_create_brief": continue_create_brief},
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
)

In [7]:
description = """
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)


Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.
"""

initial_message = f"""{description}

Is everything clear with the above description?
"""

system_message="When you receive 'Suggested function Call:' the function should be executed by Function_executor_proxy."

groupchat = autogen.GroupChat(agents=[user_proxy, product_owner, function_executor_proxy], messages=[], max_round=30)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=initial_message)
# user_proxy.initiate_chat(product_owner, message=initial_message)

User_proxy (to chat_manager):


Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)


Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.


Is everything clear with the above description?


--------------------------------------------------------------------------------
Product_owner (to chat_manager):

Do we have a specific source or API from where the application should retrieve the cryptocurrency prices?

--------------------------------------------------------------------------------
User_proxy (to chat_manager):

That's a technical decision. I leave it to you.

--------------------------

******************************************************************

--------------------------------------------------------------------------------
Product_owner (to chat_manager):

This technical brief seems to be in line with the client's requests. The application will:

1. Retrieve the current cryptocurrency prices for Bitcoin and Ethereum.
2. Use the CryptoCompare API for fetching this data.
3. Retrieve the data every 2 seconds.
4. Publish the data to the 'new_crypto_price' Kafka topic.
5. The message will be a JSON with two attributes: 'price' and 'crypto_currency'. 'price' will represent the current price of the cryptocurrency in USD, and 'crypto_currency' will represent the type of cryptocurrency.
6. Use the 'crypto_currency' attribute, utf-8 encoded, as a partition key when publishing the message.

If there are no further clarifications or changes, I believe we can proceed with this design.

--------------------------------------------------------------------------------
User_